# Test of GraphSAGE
- use DGL
- predict `graphs`
- valid, test data are in the training dataset

In [32]:
import os
import dgl
import csv
import json
import torch
import random
import subprocess
import torch as th
import numpy as np
import pandas as pd
import torch.nn as nn
import dgl.nn as dglnn
import torch.nn.functional as F

from tqdm.notebook import tqdm
from sklearn.decomposition import PCA
from torch.optim import AdamW, lr_scheduler
from dgl.nn import GraphConv, GATConv, SAGEConv
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup

- check the GPU and assign the GPU by the best memory usage
- on A100 server we use cuda:0

In [33]:
# def get_free_gpu():
#     try:
#         # Run nvidia-smi command to get GPU details
#         _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
#         command = "nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader"
#         memory_free_info = _output_to_list(subprocess.check_output(command.split())) 
#         memory_free_values = [int(x) for i, x in enumerate(memory_free_info)]
        
#         # Get the GPU with the maximum free memory
#         best_gpu_id = memory_free_values.index(max(memory_free_values))
#         return best_gpu_id
#     except:
#         # If any exception occurs, default to GPU 0 (this handles cases where nvidia-smi isn't installed)
#         return 0

# if torch.cuda.is_available():
#     # Get the best GPU ID based on free memory and set it
#     best_gpu_id = get_free_gpu()
#     device = torch.device(f"cuda:{best_gpu_id}")
# else:
#     device = torch.device("cpu")
#     print("there's no available GPU")

device = torch.device(f"cuda:{0}")
print(device)

cuda:0


## Fix the seed

In [34]:
#fix seed
def same_seeds(seed = 8787):
    torch.manual_seed(seed)
    # random.seed(seed) 
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Load the embedding

In [35]:
DIM = 50
embedding = "transH"
embedding = f'{embedding}_{DIM}'

with open(f"../../data/4_embedding/synthesize/{embedding}.vec.json", "r") as f:
    tmp = json.load(f)

index2entemb = {idx:emb for idx, emb in enumerate(tmp["ent_embeddings.weight"])}
index2relemb = {idx:emb for idx, emb in enumerate(tmp["rel_embeddings.weight"])}

In [36]:
len(index2entemb)

26868

In [37]:
len(index2relemb)

23

In [38]:
with open("../../data/source_data/before_embedding/3.10/all_graph_data.jsonl", "r") as f:
    print("Loading the data...")
#     input_data = list(f)
#     input_data = [json.loads(line) for idx, line in tqdm(f, desc="Loading")]

    # only process 40000 data from 400000 data
    wanted_data = 399000
    input_data = []
    for idx, line in tqdm(enumerate(f), total=wanted_data, desc="Loading"):
        if idx == wanted_data:
            break
        input_data.append(json.loads(line))
        
    print("FINISH...")

Loading the data...


Loading:   0%|          | 0/399000 [00:00<?, ?it/s]

FINISH...


In [39]:
len(input_data)

399000

- Convert the 'node_feat' and 'edge_attr' from int to embedding
    - takes about 45 min to transform the embedding
    - if use original method -> takes about 60 hours

In [40]:
# ============ If type(input_data[0] == dict) ============
for data_point in tqdm(input_data):
    data_point['node_feat'] = [index2entemb[node_id] for node_id in data_point['node_feat']]
    data_point['edge_attr'] = [index2relemb[edge_id] for edge_id in data_point['edge_attr']]


# ============ If type(input_data[0] == str) ============
# for idx, data in tqdm(enumerate(input_data)):
    
#     # make the data from string to int
#     data_point = json.loads(data)

#     data_point['node_feat'] = [index2entemb[node_id] for node_id in data_point['node_feat']]
#     data_point['edge_attr'] = [index2relemb[edge_id] for edge_id in data_point['edge_attr']]

#     input_data[idx] = data_point

  0%|          | 0/399000 [00:00<?, ?it/s]

In [41]:
len(input_data)

399000

In [42]:
input_data[1]['node_feat']

[[-0.13400593400001526,
  0.0745830237865448,
  0.035443343222141266,
  0.12708544731140137,
  0.1313164234161377,
  -0.11280117183923721,
  0.03557184711098671,
  0.13569022715091705,
  0.04212702810764313,
  0.015788700431585312,
  -0.12025688588619232,
  -0.03752604499459267,
  0.1454906016588211,
  0.15406882762908936,
  -0.02328800968825817,
  0.04003113880753517,
  0.044135212898254395,
  -0.033310115337371826,
  0.07393254339694977,
  -0.01879263110458851,
  0.06671767681837082,
  -0.054439689964056015,
  -0.09587514400482178,
  0.11176080256700516,
  -0.02801632322371006,
  -0.08088016510009766,
  0.00047018786426633596,
  -0.13521791994571686,
  0.010783975943922997,
  0.022552018985152245,
  0.036946117877960205,
  -0.0462837815284729,
  -0.03688304126262665,
  0.029622813686728477,
  -0.07293407618999481,
  0.06105346232652664,
  -0.00059130834415555,
  -0.11954065412282944,
  -0.11781751364469528,
  -0.051076941192150116,
  0.1166776791214943,
  -0.02639642171561718,
  -0.0

In [43]:
input_data[0]['labels']

[0, 0, 0, 0, 0, 0, 0]

## Data Loader

In [44]:
class GraphDataset(Dataset):
    def __init__(self, data_list, device):
        self.data_list = data_list
        self.device = device
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data = self.data_list[idx]
        return data

def collate(samples):
    data_list = samples
    batched_graphs = []
    for data in data_list:
        g = dgl.graph((th.tensor(data["edge_index"][0]), th.tensor(data["edge_index"][1])), num_nodes=data["num_nodes"])

        g.ndata['feat'] = th.tensor(data["node_feat"])
        g.edata['feat'] = th.tensor(data["edge_attr"])

        g.edata['label'] = th.tensor(data['labels'])

        batched_graphs.append(g)
    
    return dgl.batch(batched_graphs)

In [45]:
total_data = len(input_data)

test_size = int(total_data * 0.1)
train_valid_size = total_data - test_size

train_valid_data = input_data[:train_valid_size]
test_data = input_data[train_valid_size:]

train_data, valid_data = train_test_split(train_valid_data, test_size=0.25, random_state=42)

# creating GraphDataset
dataset_data = {
    'train': GraphDataset(train_data, device),
    'valid': GraphDataset(valid_data, device),
    'test': GraphDataset(test_data, device)
}

print("Datasets loaded and ready for training!")

Datasets loaded and ready for training!


In [46]:
dataset_data['train'][10]['labels']

[0, 0, 0]

In [47]:
len(dataset_data['train'])

269325

- choose batch size

In [48]:
def create_dataloaders(batch_size, shuffle=True):
    dataloaders = {}
    for dataset_name, dataset in dataset_data.items():
        # do not shuffle the testing dataset
        if dataset_name == "test":
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)    
        else:
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate)
    return dataloaders

dataloaders = create_dataloaders(200)

- Turn the print message to a log file

In [49]:
import datetime

now = datetime.datetime.now()

formatted_time = now.strftime("%m%d_%H:%M")

log_file_path = f"./log_message/{formatted_time}_GraphSAGE_{embedding}.log"

def add_log_msg(msg, log_file_path=log_file_path):
    with open(log_file_path, 'a') as f:
        f.write(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}\n')
    print(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}')

print(log_file_path)

./log_message/0125_02:42_GraphSAGE_transH_50.log


### Model

In [50]:
class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(GraphSAGE, self).__init__()
        self.layer1 = dglnn.SAGEConv(in_dim, hidden_dim, 'pool')
        self.layer2 = dglnn.SAGEConv(hidden_dim, out_dim, 'pool')
        self.dropout = nn.Dropout(0.25)

    def forward(self, g, inputs):
        h = self.layer1(g, inputs)
        h = torch.relu(h)
        h = self.dropout(h)
        h = self.layer2(g, h)
        return h

In [51]:
class MLPPredictor(nn.Module):
    def __init__(self, out_feats, out_classes):
        super().__init__()
        self.W = nn.Linear(out_feats*2, out_classes)

    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        score = self.W(torch.cat([h_u, h_v], 1))
        return {'score': score}

    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']

In [52]:
class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, num_classes):
        super().__init__()
        self.sage = GraphSAGE(in_features, hidden_features, out_features)
        self.pred = MLPPredictor(out_features, num_classes)
      
    def forward(self, g, node_feat, return_logits=False):
        h = self.sage(g, node_feat)
        logits = self.pred(g, h)
        
        return logits

- Model Forward  

In [53]:
def model_fn(batched_g, model, criterion, device, count=1, which_type='train'):
    """Forward a batch through the model."""
#     batched_g, labels = data
    batched_g = batched_g.to(device)
    
    labels = batched_g.edata['label'].to(device)
    
    logits = model(batched_g, batched_g.ndata['feat'].float())

    loss = criterion(logits, labels)

    output = torch.softmax(logits, dim=1)
    preds = output.argmax(1)
    
    # Compute accuracy
    accuracy = torch.mean((preds == labels).float())
        
    return loss, accuracy, preds

### Traning Loop

In [54]:
# # For release the GPU memory
# # No need to restart the kernel

# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [55]:
seed = 5269
in_dim = DIM # dimension of the node feature
hidden_dim = 64
out_dim = 128
num_classes = 2 # for DARPA

lr = 5e-4

total_steps = 100
patience = 5
waiting = 0

In [56]:
model = Model(in_dim, hidden_dim, out_dim, num_classes)
best_model_path = f"./checkpoint_graphSAGE/best_model_GraphSAGE_{embedding}.pt"

optimizer = AdamW(model.parameters(), lr)

scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0, last_epoch=-1, verbose=False)

criterion = nn.CrossEntropyLoss()
# criterion = torch.nn.BCEWithLogitsLoss()

In [57]:
same_seeds(seed)
model = model.to(device)
best_val_loss = float('inf')

# Training Part
for epoch in tqdm(range(total_steps)):
    # Train
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    
    for batched_g in tqdm(dataloaders['train'], desc="Training", position=0, leave=True):
        num_batches += 1
        loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='train')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()

    scheduler.step()
    add_log_msg(f"total batches: {num_batches}")

    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches

    add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

    
    # Validation Part
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0


    with torch.no_grad():
        for batched_g in tqdm(dataloaders['valid'], desc="Validation", position=0, leave=True):
            loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='validation')
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    current_loss = total_loss / num_batches
    
    add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')
    
            
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        waiting = 0
        
        if os.path.exists(best_model_path):
            os.remove(best_model_path)
            add_log_msg("Find a better model!!")

        torch.save(model.state_dict(), best_model_path)
 
    else:
        waiting += 1
        if waiting >= patience:
            add_log_msg("============================== Early stopping ==================================")
            break

  0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/1347 [00:00<?, ?it/s]

01/25/2024, 03:12:18# total batches: 1347
01/25/2024, 03:12:18# Epoch 0 | Train Loss: 0.1038 | Train Accuracy: 0.9693


Validation:   0%|          | 0/449 [00:00<?, ?it/s]

01/25/2024, 03:25:23# Validation Loss: 0.0678 | Validation Accuracy: 0.9773

01/25/2024, 03:25:23# Find a better model!!


...

01/25/2024, 11:56:28# Validation Loss: 0.0251 | Validation Accuracy: 0.9871



Training:   0%|          | 0/1347 [00:00<?, ?it/s]

01/25/2024, 12:20:16# total batches: 1347
01/25/2024, 12:20:16# Epoch 33 | Train Loss: 0.0254 | Train Accuracy: 0.9870


Validation:   0%|          | 0/449 [00:00<?, ?it/s]

01/25/2024, 12:28:08# Validation Loss: 0.0251 | Validation Accuracy: 0.9871

01/25/2024, 12:28:08# ============================== Early stopping ==================================


### Testing Loop

In [58]:
# load the pretrained model
model.load_state_dict(torch.load(best_model_path))

model.to(device)
model.eval()

total = 0
correct = 0
count = 0

true_labels = []
predicted_labels = []

with torch.no_grad():
    for batched_g in tqdm(dataloaders['test'], desc="Testing", position=0, leave=True):
#         print(f"data:{data[1]}")
        loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, count, which_type='test')
        labels = batched_g.edata['label'].to(device)
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())
            
        count += 1
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

add_log_msg(f'Test Accuracy: {100 * correct / total} %\n\n\n')

Testing:   0%|          | 0/200 [00:00<?, ?it/s]

01/25/2024, 12:31:50# Test Accuracy: 98.70279885247591 %





In [59]:
report_data = classification_report(true_labels, predicted_labels, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

report_df

,precision,recall,f1-score,support
0,0.990904,0.995711,0.993301,1.781100e+07
1,0.859119,0.741053,0.795730,6.287000e+05
accuracy,0.987028,0.987028,0.987028,9.870280e-01
macro avg,0.925011,0.868382,0.894516,1.843970e+07
weighted avg,0.986411,0.987028,0.986565,1.843970e+07


### Training

- Fix the seed and save the model.state_dict that contains the initial weight

In [67]:
# seed = 8787
# same_seeds(seed)

# model = Model(in_features=50, hidden_features=64, out_features=128, num_classes=167)
# torch.save(model.state_dict(), 'model3_initial(graphsage)/initial_weight.pth')

In [33]:
# # model.layer1.fc_self.weight
# model.sage.layer1.fc_self.weight

- Check if model really load the model_dict

In [34]:
# model = Model(in_features=50, hidden_features=64, out_features=128, num_classes=167)
# model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))
# model.sage.layer1.fc_self.weight

- For release the GPU memory
    - no need to restart the kernel

In [35]:
# # For release the GPU memory
# # No need to restart the kernel

# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
# # ======================================== handlig the output excel files ========================================
# mapping_file = './new_mapping.txt'
# label_mapping = {}
# with open(mapping_file, 'r') as f:
#     for line in f:
#         parts = line.strip().split(': ')
#         label_mapping[int(parts[1])] = parts[0]
        
# # 将映射后的标签应用到true和predicted标签列表
# mapped_true_labels = [label_mapping[label] for label in true_labels]
# mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

# # 生成Scikit-learn报告信息的DataFrame
# report_data = classification_report(mapped_true_labels, mapped_predicted_labels, output_dict=True)
# report_df = pd.DataFrame(report_data).transpose()

# # mapped_true_labels_np = np.array(mapped_true_labels)
# # mapped_predicted_labels_np = np.array(mapped_predicted_labels)

# # print("mapped_true_labels 的形状:", mapped_true_labels_np.shape)
# # print("mapped_predicted_labels 的形状:", mapped_predicted_labels_np.shape)

# report_folder = 'classification_report'
# os.makedirs(report_folder, exist_ok=True)

# count = 0
# while True:
#     report_filename = f'classification_report-transE_50-graphSAGE-{count}.xlsx'
#     labels_filename = f'mapped_true_predicted_labels-transE_50-graphSAGE-{count}.xlsx'
    
#     report_path = os.path.join(report_folder, report_filename)
#     labels_path = os.path.join(report_folder, labels_filename)
    
#     if not os.path.exists(report_path) and not os.path.exists(labels_path):
#         break
#     count += 1

    
# report_df.to_excel(report_path, index_label='Label')

# mapped_labels_df = pd.DataFrame({'true_label': mapped_true_labels, 'predicted_label': mapped_predicted_labels})
# mapped_labels_df.to_excel(labels_path, index=False)

# add_log_msg(f"report path: {report_path}")
# add_log_msg(f"label path: {labels_path}")

# mapped_report = classification_report(mapped_true_labels, mapped_predicted_labels)
# add_log_msg(f"mapped_report:\n{mapped_report}")